In [0]:
-- create sink table using schema inferred from source table
-- column mapping is enabled to accomodate characters that are not allowed by parquet
-- https://docs.databricks.com/aws/en/delta/column-mapping

declare or replace qry_str string;
declare or replace col_list string;

create catalog if not exists identifier(:tgt_catalog);
use catalog identifier(:tgt_catalog);
create schema if not exists identifier(:tgt_schema);
use schema identifier(:tgt_schema);

set var col_list = (
  select
    concat_ws(
      ', ',
      collect_list(concat('`', col_name, '`', ' ', col_type))
    ) as column_list
  from (
    select
      col.name       as col_name,
      col.type.name  as col_type
    from (
      select
        explode(
          from_json(
            json_str,
            'struct<columns: array<struct<name: string, type: struct<name: string>>>>'
          ).columns
        ) as col
      from (values (:src_tbl_metadata)) as t(json_str)
    )
  )
);

set var qry_str = 
  'create table if not exists ' || :tgt_table || ' (' || col_list || ')'
  'tblproperties('
    '\'delta.columnMapping.mode\' = \'name\''
  ')';

execute immediate qry_str;

In [0]:
-- Enable liquid clustering if clustering columns are specified
begin
  declare qry_str string;

  if :sink_cluster_cols != '' then
    set qry_str = 
      "alter table " || :tgt_catalog || '.' || :tgt_schema || '.' || :tgt_table || " cluster by (" || :sink_cluster_cols || ")";
    execute immediate qry_str;
  end if;
end

In [0]:
-- Enable Iceberg reads
-- https://docs.databricks.com/aws/en/delta/uniform#enable-iceberg-reads-on-an-existing-table
begin
  declare qry_str string;
  declare enable_iceberg_reads boolean;
  set enable_iceberg_reads = cast(:enable_iceberg_reads as boolean);

  if enable_iceberg_reads is true then
    -- Disable deletion vectors
    set qry_str =
      'alter table ' || :tgt_catalog || '.' || :tgt_schema || '.' || :tgt_table || ' set tblproperties(\'delta.enableDeletionVectors\' = false);';
    execute immediate qry_str;
    
    -- Enable Iceberg reads
    set qry_str = 
      'alter table ' || :tgt_catalog || '.' || :tgt_schema || '.' || :tgt_table || ' set tblproperties('
        '\'delta.columnMapping.mode\' = \'name\','
        '\'delta.enableIcebergCompatV2\' = \'true\','
        '\'delta.universalFormat.enabledFormats\' = \'iceberg\')';
    execute immediate qry_str;
  end if;
end